ACTOR 2 CRITIC

Logan Wong

law3082

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
!ls -la

/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
total 30
drwx------ 2 root root 4096 Sep 25 15:17 checkpoints
drwx------ 2 root root 4096 Sep 25 15:17 code
drwx------ 2 root root 4096 Sep 25 15:04 .git
-rw------- 1 root root 6380 Oct 16 04:07 github_terminal.ipynb
-rw------- 1 root root   33 Sep 26 19:25 .gitignore
drwx------ 2 root root 4096 Sep 25 15:17 models
-rw------- 1 root root 2348 Sep 29 02:21 README.md
drwx------ 2 root root 4096 Sep 25 15:17 results


In [3]:
!pip install gymnasium[atari,accept-rom-license] ale-py sb3_contrib stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 21.6 MB/s eta 0:00:00


In [4]:
import os
import torch
import gymnasium as gym
import stable_baselines3
import ale_py
import numpy as np

# RL Algorithm
from stable_baselines3 import A2C


# Visualization
from PIL import Image
import io
import base64
from IPython.display import display, HTML



# For debugging
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback
import time

# Action masking
from gymnasium import ActionWrapper
from stable_baselines3.common.atari_wrappers import AtariWrapper

# Vector environment
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecEnvWrapper, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

import gc

print("All imports working")

All imports working


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

GPU available: True
GPU: Tesla T4


In [6]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [7]:
class ActionReducer(ActionWrapper):
  def __init__(self, env):
    super().__init__(env)

    # NOOP, FIRE, UP, and DOWN only. No UPFIRE. No DOWNFIRE.
    self.allowed_actions = [0,1,2,3]

    self.action_space = gym.spaces.Discrete(len(self.allowed_actions))

  def action(self, action):
    return self.allowed_actions[action]

In [8]:
def make_env():
  env = gym.make("ALE/Bowling-v5")
  env = ActionReducer(env)
  env = Monitor(env)
  # disable reward clipping
  env = AtariWrapper(env, clip_reward=False)
  return env

In [9]:
seed = 316
torch.manual_seed(seed)

# env = DummyVecEnv([make_env])
# env = VecFrameStack(env, n_stack=4)


# env = make_vec_env("ALE/Bowling-v5", n_envs=4)
num_of_parallel_envs = 4
env = DummyVecEnv([make_env for _ in range(num_of_parallel_envs)])  # Parallel vector environments
env = VecFrameStack(env, n_stack=4)

# GOAL: Compare A2C to Baseline DQN.
# But... these 2 algorithms are pretty different
# They only share 4 parameters: learning rate, gamma, verbose, and device
# So really, only 2 parameters that matter...
# Learning rate = 0.0001
# gamma = 0.99

model = A2C(
    "CnnPolicy",
    env,
    learning_rate=0.0001,
    gamma=0.99,
    verbose=1,
    device="cuda",

    n_steps=5,              # Steps per update; like DQN's train_freq=4
    gae_lambda=0.95,        # Standard value; provides slight bias for lower variance
    ent_coef=0.01,          # exploration (like epsilon-greedy)
    vf_coef=0.5,            # Standard value; balance btwn actor & critic loss
    max_grad_norm=0.5,      # Prevents exploding gradients
    use_rms_prop=True       # RMSprop works well for Atari
)


# gamma=0.999,          # Longer horizon for sparse rewards
# n_steps=10,           # More steps per update
# ent_coef=0.1,         # Much more exploration

print("Actor 2 Critic model created")

Using cuda device
Wrapping the env in a VecTransposeImage.
Actor 2 Critic model created


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
class SimpleCheckpointCallback(BaseCallback):
    def __init__(self, save_freq, save_path, name_prefix):
        super().__init__()
        self.save_freq = save_freq
        self.save_path = save_path
        self.name_prefix = name_prefix

    def _on_step(self):
        if self.n_calls % self.save_freq == 0:
            path = os.path.join(self.save_path, f"{self.name_prefix}_{self.n_calls}")
            self.model.save(path)
            print(f"Checkpoint saved at step {self.n_calls}")
        return True

In [11]:
total_timesteps = 10000000    # 10M
# total_timesteps = 5000000    # 5M
# total_timesteps =  2000000    # 2M
# total_timesteps =  1000000    # 1M
# total_timesteps =   100000    # 100K
# total_timesteps =    10000    # 10K
# total_timesteps =     5000    # 5K

# Save couple of steps
checkpoint_callback = CheckpointCallback(
    save_freq=100000,
    save_path="/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari/checkpoints",
    name_prefix=f"a2c_{total_timesteps}"
)

# Time how long it takes
print("Training started")
start_time = time.time()

# log_interval: Print train metrics every 10 episodes
model.learn(
    total_timesteps=total_timesteps,
    callback=checkpoint_callback,
    progress_bar=True,
    log_interval=10
)
end_time = time.time()
print("Training done")

env.close()

# Calculate run time
training_duration = end_time - start_time
time_in_minutes_and_seconds = convert(training_duration)
print(f"Time taken: {time_in_minutes_and_seconds}")
# print(f"Speed: {total_timesteps/training_duration:.2f} steps/second")

# Save model to Google Drive
trained_model_save_path = f"/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari/models/a2c_{total_timesteps}"
model.save(trained_model_save_path)
print("Model saved to Google Drive")

Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

Streaming output truncated to the last 5000 lines.
|    value_loss         | 2.97e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.31e+03 |
|    ep_rew_mean        | 27.4     |
| time/                 |          |
|    fps                | 280      |
|    iterations         | 497070   |
|    time_elapsed       | 35496    |
|    total_timesteps    | 9941400  |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | -0.00333 |
|    learning_rate      | 0.0001   |
|    n_updates          | 497069   |
|    policy_loss        | 0.466    |
|    value_loss         | 2.44     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.31e+03 |
|    ep_rew_mean        | 27.4     |
| time/                 |          |
|    fps                | 280      |
|    iterations         

Training done
Time taken: 9:55:06
Model saved to Google Drive


In [12]:
torch.cuda.empty_cache()
del model
del env